In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from pathlib import Path
import joblib

In [7]:
df_base = pd.read_csv('/Users/pm/Documents/GitHub/ML_Project_1/data/processed/df_wdi_clean_base.csv')

In [8]:
id_cols = ["country", "country_code", "year"]

# last look at the cleaned dataframe
print("Columns:", list(df_base.columns))   
print("Shape:", df_base.shape)
print("\nTypes :")
print(df_base.dtypes)

print("\nNA par colonne (desc) :")
print(df_base.isna().sum().sort_values(ascending=False))

print("\nColonnes non numériques (hors id) :")
non_num = (df_base.columns.difference(id_cols)
           .difference(df_base.select_dtypes(include=["number","bool"]).columns))
print(list(non_num))

Columns: ['country_code', 'year', 'Access to electricity (% of population)', 'Current health expenditure (% of GDP)', 'Current health expenditure per capita (current US$)', 'Government expenditure on education, total (% of GDP)', 'Hospital beds (per 1,000 people)', 'Immunization, DPT (% of children ages 12-23 months)', 'Literacy rate, adult total (% of people ages 15 and above)', 'Mortality rate, infant (per 1,000 live births)', 'School enrollment, primary and secondary (gross), gender parity index (GPI)', 'health_exp_share', 'log1p_health_exp_pc', 'GDP_pc_imputed', 'y_t1', 'Mortality rate, infant (per 1,000 live births)_lag1', 'Access to electricity (% of population)_lag1', 'Current health expenditure (% of GDP)_lag1', 'Current health expenditure per capita (current US$)_lag1', 'GDP_pc_imputed_lag1', 'Government expenditure on education, total (% of GDP)_lag1', 'Hospital beds (per 1,000 people)_lag1', 'Immunization, DPT (% of children ages 12-23 months)_lag1', 'Literacy rate, adult to

In [9]:
# convert features to numeric, coercing errors to NaN
feat = df_base.columns.difference(id_cols)

df_base[feat] = (df_base[feat]
                      .replace({r",": ""}, regex=True)  # supprime séparateurs de milliers
                      .apply(pd.to_numeric, errors="coerce"))

# Colonnes qui ont généré des NaN après conversion (suspects)
suspects = df_base[feat].isna().sum()
print(suspects[suspects > 0].sort_values(ascending=False))

Literacy rate, adult total (% of people ages 15 and above)_yoy_yoy                     2662
School enrollment, primary and secondary (gross), gender parity index (GPI)_yoy_yoy    2355
Hospital beds (per 1,000 people)_yoy_yoy                                               1767
Access to electricity (% of population)_yoy_yoy                                        1723
Immunization, DPT (% of children ages 12-23 months)_yoy_yoy                            1592
                                                                                       ... 
Literacy rate, adult total (% of people ages 15 and above)                               30
Literacy rate, adult total (% of people ages 15 and above)_roll3                         30
Mortality rate, infant (per 1,000 live births)                                           15
School enrollment, primary and secondary (gross), gender parity index (GPI)              15
School enrollment, primary and secondary (gross), gender parity index (GPI)_roll

In [10]:
# checking that all features (except IDs) are numeric
assert df_base[feat].select_dtypes(exclude=["number","bool"]).empty, \
       f"Non numériques: {list(df_base[feat].select_dtypes(exclude=['number','bool']).columns)}"

In [11]:
# make sure ID columns are of appropriate types

id_cols = [c for c in ["country", "country_code", "year"] if c in df_base.columns]
if "year" in df_base.columns:
    df_base["year"] = pd.to_numeric(df_base["year"], errors="coerce").astype("Int64")

In [12]:
# Both columns are guaranteed to exist and be numeric.(che = current health expenditure)
che_pct = "Current health expenditure (% of GDP)"
che_pc  = "Current health expenditure per capita (current US$)"


# convert % -> share in [0,1]
df_base["health_exp_share"] = df_base[che_pct] / 100.0

# stabilize heavy-tailed money values (safe: no log(0) since che_pc > 0)
df_base["log1p_health_exp_pc"] = np.log1p(df_base[che_pc])

# GDP_pc = 100 * (che_pc / (che_pct / 100)) = 100 * che_pc / che_pct
# only if che_pct > 0 and che_pc is not NaN
mask = df_base[che_pct].gt(0) & df_base[che_pc].notna()
df_base["GDP_pc_imputed"] = np.where(
    mask,
    100 * df_base[che_pc] / df_base[che_pct],
    np.nan
)

In [13]:

# Ensure year numeric for sorting/splits

df_base["year"] = pd.to_numeric(df_base["year"], errors="coerce").astype("Int64")

target_col = "Mortality rate, infant (per 1,000 live births)"
assert target_col in df_base.columns, "Target missing."

df_base = df_base.sort_values(["country_code","year"])
df_base["y_t1"] = df_base.groupby("country_code")[target_col].shift(-1)

In [15]:
# 0) Make sure rows are ordered within each country for time ops
df_base = df_base.sort_values(["country_code", "year"]).copy()

# 1) AR(1) of the target (single insert is fine)
df_base[f"{target_col}_lag1"] = (
    df_base.groupby("country_code", observed=True)[target_col].shift(1)
)

# 2) Define base features from ORIGINAL numeric cols (no derived ones)
#    Exclude identifiers and the target; also exclude any existing *_lag1/_yoy just in case
num_now = df_base.select_dtypes(include=["number"]).columns
base_feats = [c for c in num_now
              if c not in {"year", target_col}
              and not (str(c).endswith("_lag1") or str(c).endswith("_yoy"))]

# 3) Compute ALL lag-1 columns at once (no loop)
lag_df = (df_base.groupby("country_code", observed=True)[base_feats]
                 .shift(1)
                 .add_suffix("_lag1"))

# 4) Compute ALL YoY % changes at once, with no implicit ffill (safer for forecasting)
yoy_df = (df_base.groupby("country_code", observed=True)[base_feats]
                 .pct_change(fill_method=None)
                 .replace([np.inf, -np.inf], np.nan)
                 .add_suffix("_yoy"))

# 5) Single concat to avoid fragmentation; then defragment with a copy
df_base = pd.concat([df_base, lag_df, yoy_df], axis=1)
df_base = df_base.copy()  # defragment the underlying blocks

In [16]:
# Final shapes
id_cols = [c for c in ["country","country_code","year"] if c in df_base.columns]
mask = df_base["y_t1"].notna()

X_cols = df_base.columns.difference(id_cols + [target_col, "y_t1"])
X = df_base.loc[mask, X_cols]
y = df_base.loc[mask, "y_t1"]

print("Panel shapes -> X:", X.shape, "| y:", y.shape)

Panel shapes -> X: (3024, 420) | y: (3024,)


In [29]:
# === POINT A: Reduce X to robust features ===
# Keep only: *_lag1, *_roll3, plus a few static engineered features if present.
static_keep = [c for c in ["health_exp_share", "log1p_health_exp_pc", "GDP_pc_imputed"] if c in X.columns]

X_reduced = pd.concat([
    X.loc[:, [c for c in X.columns if c.endswith("_lag1")]],
    X.loc[:, [c for c in X.columns if c.endswith("_roll3")]],
    X.loc[:, static_keep]
], axis=1)

# Drop any accidental duplicate column names (can happen after multiple runs)
X_reduced = X_reduced.loc[:, ~X_reduced.columns.duplicated()]

print("Reduced X shape:", X_reduced.shape)

Reduced X shape: (3024, 149)


In [30]:
years = sorted(df_base.loc[mask, "year"].dropna().unique())
splits = []
for i in range(1, len(years)):
    val_year  = years[i]          # validate on year t
    train_max = years[i-1]        # train on years <= t-1
    tr_idx = X.index[df_base.loc[mask, "year"] <= train_max]
    va_idx = X.index[df_base.loc[mask, "year"] == val_year]
    if len(tr_idx) and len(va_idx):
        splits.append((val_year, tr_idx, va_idx))

print(f"{len(splits)} time splits:", years)

13 time splits: [np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]


In [33]:
def safe_rmse(y_true, y_pred):
    """Compute RMSE, compatible with older scikit-learn (fallback to sqrt(MSE))."""
    try:
        from sklearn.metrics import root_mean_squared_error
        return float(root_mean_squared_error(y_true, y_pred))
    except Exception:
        return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def align_non_null(y_true, y_pred):
    """Align by index, then drop rows where either side is NaN."""
    yt, yp = y_true.align(y_pred, join="inner")
    m = yt.notna() & yp.notna()
    return yt[m], yp[m]

In [34]:
pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("rf", RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1, min_samples_leaf=3)),
])

for (val_year, tr, va) in splits:
    cols_keep = X_reduced.columns[X_reduced.loc[tr].notna().any(axis=0)]
    if len(cols_keep) == 0:
        continue

    X_tr, y_tr = X_reduced.loc[tr, cols_keep], y.loc[tr]
    X_va, y_va = X_reduced.loc[va, cols_keep], y.loc[va]

    pipe.fit(X_tr, y_tr)
    y_pred = pipe.predict(X_va)

    yt, yp = align_non_null(y_va, pd.Series(y_pred, index=X_va.index))
    if len(yt) == 0:
        continue

    rmse_ml = safe_rmse(yt, yp)
    metrics_df.loc[metrics_df.year.eq(int(val_year)), "rmse_ml"] = rmse_ml

# Compare medians on the same set of validation years
df_eval = metrics_df.dropna(subset=["rmse_ml"]).sort_values("year")
med_naive = float(np.median(df_eval.rmse_naive))
med_ml    = float(np.median(df_eval.rmse_ml))
print(df_eval[["year","rmse_naive","rmse_ml"]])
print(f"Corrected median RMSE — naive: {med_naive:.3f} | ML: {med_ml:.3f} | Gain: {med_naive - med_ml:.3f}")

    year  rmse_naive   rmse_ml
0   2011    3.015185  6.418056
1   2012    8.888874  3.955585
2   2013    4.254741  4.274695
3   2014    5.469812  5.162245
4   2015    3.968236  4.558003
5   2016    2.720694  4.139558
6   2017    5.173034  3.263281
7   2018    7.565400  3.953295
8   2019    2.110237  2.554821
9   2020    2.543783  1.662435
10  2021    7.988861  7.390196
11  2022    2.684549  3.133115
12  2023    8.402973  5.064932
Corrected median RMSE — naive: 4.255 | ML: 4.140 | Gain: 0.115


In [39]:
# --- Final fit (already done above) ---
# pipe.fit(X_reduced, y)

# --- Build X_future robustly: dedupe columns, then align to training feature order ---
last_year   = int(df_base["year"].max())
future_mask = (df_base["year"] == last_year)   # do NOT filter with y_t1 here

# 1) Freeze the exact training feature order
try:
    expected_cols = list(pipe.feature_names_in_)   # sklearn >= 1.0
except Exception:
    expected_cols = list(X_reduced.columns)        # fallback

# 2) Take last-year rows and drop duplicate column names (keep first occurrence)
future_raw = df_base.loc[future_mask].copy()
future_raw = future_raw.loc[:, ~future_raw.columns.duplicated()]

# (Optional, extra safety) If duplicates still exist for any reason, collapse by first:
# future_raw = future_raw.T.groupby(level=0).first().T

# 3) Reindex to the exact expected feature set and order
#    - Missing columns will be added as NaN (the pipeline imputer will handle them).
#    - Extra columns are dropped.
X_future = future_raw.reindex(columns=expected_cols)

# 4) Predict next year
pred_next = pipe.predict(X_future)

# 5) Package predictions
pred_df = (df_base.loc[future_mask, id_cols]
           .assign(year_input=last_year,
                   year_pred=last_year + 1,
                   y_pred=pred_next,
                   target_last=df_base.loc[future_mask, target_col].values))

print(pred_df.head())

   country_code  year  year_input  year_pred     y_pred  target_last
14          ABW  2024        2024       2025   4.751747         4.75
29          AFG  2024        2024       2025  59.032847        61.35
44          AGO  2024        2024       2025  47.439405        47.95
59          ALB  2024        2024       2025   8.547565         8.05
74          AND  2024        2024       2025   3.040339         3.15


In [42]:
# Save metrics, predictions, model, and feature list
ROOT = Path("..").resolve()

PATH_METRICS = ROOT / "docs" / "metrics_backtest.csv"
PATH_PREDS   = ROOT / "data" / "processed" / "preds_next_year.csv"
PATH_MODEL   = ROOT / "models" / "trained_model.pkl"
PATH_FEATS   = ROOT / "data" / "processed" / "feature_cols_training.txt"

# Create folders if they don't exist
for p in [PATH_METRICS.parent, PATH_PREDS.parent, PATH_MODEL.parent, PATH_FEATS.parent]:
    p.mkdir(parents=True, exist_ok=True)
# export predictions
assert 'metrics_df' in globals(), "metrics_df is missing. Run your backtest cell first."
metrics_df.to_csv(PATH_METRICS, index=False, encoding="utf-8-sig")
print(f"Saved backtest metrics → {PATH_METRICS}")

# save feature list
assert 'pred_df' in globals() and not pred_df.empty, "pred_df is missing or empty. Build it before exporting."
pred_df.to_csv(PATH_PREDS, index=False, encoding="utf-8-sig")
print(f"Saved next-year predictions → {PATH_PREDS}")
# export feature list
assert 'pipe' in globals(), "Trained model 'pipe' not found. Fit it or assign your model to 'pipe'."
joblib.dump(pipe, PATH_MODEL)
print(f"Saved trained model → {PATH_MODEL}")
# export feature list in exact training order
try:
    feature_names = list(pipe.feature_names_in_)   # sklearn >= 1.0
except Exception:
    # Fallback: use the columns you trained on (prefer X_reduced if that's what you used)
    if 'X_reduced' in globals():
        feature_names = list(X_reduced.columns)
    elif 'X' in globals():
        feature_names = list(X.columns)
    else:
        raise RuntimeError("Cannot infer training feature names. Provide X_reduced or X.")

pd.Series(feature_names).to_csv(PATH_FEATS, index=False, header=False)
print(f"Saved training feature list → {PATH_FEATS}")

Saved backtest metrics → /Users/pm/Documents/GitHub/ML_Project_1/docs/metrics_backtest.csv
Saved next-year predictions → /Users/pm/Documents/GitHub/ML_Project_1/data/processed/preds_next_year.csv
Saved trained model → /Users/pm/Documents/GitHub/ML_Project_1/models/trained_model.pkl
Saved training feature list → /Users/pm/Documents/GitHub/ML_Project_1/data/processed/feature_cols_training.txt
